In [ ]:
import schiller_lab_tools
import glob
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

from scipy.cluster.vq import kmeans, whiten
from scipy.ndimage import gaussian_filter, distance_transform_edt
from scipy.optimize import curve_fit
from skimage import measure, transform

from numba import njit

from scipy.signal import convolve

from skimage import measure, filters
import pyvista as pv

# Data import

## Helper functions

In [ ]:
wd = schiller_lab_tools.read_hdf5(glob.glob("wd*.h5")[0])
od = schiller_lab_tools.read_hdf5(glob.glob("od*.h5")[0])
md, t = schiller_lab_tools.read_asc(glob.glob("md-cfg*.asc")[0])
md = md.sort_values(by = ['p_id'])
md = md.reset_index()
md = md.drop(['index'], axis = 1)
boxDims = wd.shape

fill_od = schiller_lab_tools.microstructure_analysis.fill(od)
fill_wd = schiller_lab_tools.microstructure_analysis.fill(wd)

# phi_fill = (fill_od - fill_wd)/(fill_od + fill_wd)
phi_fill = fill_od - fill_wd

# phi = od - wd
# phi_fill = schiller_lab_tools.microstructure_analysis.fill(phi)

In [ ]:
def radial_distribution_function(L, positions, positions2 = None, binsize = None):
    r_max = np.ceil(np.sqrt(3)*L)
    Vsphere = 4/3*np.pi*(r_max**3)

    if binsize is None:
        bins = np.linspace(0, r_max, int(r_max) + 1) 
        slc = slice(0, int(np.ceil(np.sqrt(3)/4*L)))
    else:
        bins = np.arange(0, r_max, binsize)
        slc = slice(0, int(np.ceil(np.sqrt(3)/4*L)//binsize))

    if positions2 is None:
        N = positions.shape[0]/Vsphere
        delta = positions[:, np.newaxis] - positions
    else:
        N = (positions.shape[0] + positions2.shape[0])/Vsphere
        delta = positions[:, np.newaxis] - positions2
    
    delta = np.where(delta > L//2, delta - L, delta)
    euclid_dist = np.linalg.norm(delta, axis = -1)
    euclid_dist = np.unique(euclid_dist)[1:]
    
    hist, bin_edges = np.histogram(euclid_dist, bins = bins)
    volume_slice = 4/3*np.pi*(np.power(bin_edges[1:], 3) - np.power(bin_edges[:-1], 3))

    x = bin_edges[slc]
    y = hist/(N*volume_slice)
    y = y[slc]
    y /= y[-1]

    return x, y

In [ ]:
import numpy as np
from scipy.optimize import minimize

def ellipsoid_surface_constraint(x, center, R, Q):
    """ Ensures that a point x lies on the ellipsoid defined by center, R (radii), and Q (rotation). """
    x_local = np.linalg.inv(Q) @ (x - center)  # Transform to local ellipsoid coordinates
    return (x_local[0] / R[0])**2 + (x_local[1] / R[1])**2 + (x_local[2] / R[2])**2 - 1

def rotation_matrix_from_vector(v):
    """ Constructs a rotation matrix from an orientation unit vector v. """
    v = v / np.linalg.norm(v)
    # Create an arbitrary perpendicular vector
    if np.abs(v[0]) < 0.9:
        perp = np.array([1, 0, 0])
    else:
        perp = np.array([0, 1, 0])
    
    v1 = np.cross(v, perp)
    v1 /= np.linalg.norm(v1)
    v2 = np.cross(v, v1)
    return np.column_stack([v1, v2, v])

def closest_ellipsoid_distance(center1, orientation1, Rp1, Ro1, center2, orientation2, Rp2, Ro2):
    """ Finds the closest distance between two ellipsoids. """
    
    # Define radii and rotation matrices for each ellipsoid
    R1 = np.array([Rp1, Ro1, Ro1])
    R2 = np.array([Rp2, Ro2, Ro2])
    
    Q1 = rotation_matrix_from_vector(orientation1)
    Q2 = rotation_matrix_from_vector(orientation2)
    
    # Initial guess: linearly interpolate along the center-to-center vector
    direction = center2 - center1
    direction /= np.linalg.norm(direction)
    x1_guess = center1 + R1[0] * direction  # Approximate surface point on E1
    x2_guess = center2 - R2[0] * direction  # Approximate surface point on E2

    # Optimization: minimize the distance between x1 and x2
    def objective(vars):
        x1, x2 = vars[:3], vars[3:]
        return np.sum((x2 - x1) ** 2)  # Squared Euclidean distance

    # Constraints: points must remain on the ellipsoid surfaces
    constraints = [
        {'type': 'eq', 'fun': lambda vars: ellipsoid_surface_constraint(vars[:3], center1, R1, Q1)},
        {'type': 'eq', 'fun': lambda vars: ellipsoid_surface_constraint(vars[3:], center2, R2, Q2)}
    ]

    # Run optimization
    result = minimize(objective, np.concatenate([x1_guess, x2_guess]), constraints=constraints, method='SLSQP')
    
    if result.success:
        x1_opt, x2_opt = result.x[:3], result.x[3:]
        min_distance = np.linalg.norm(x2_opt - x1_opt)
        return min_distance, x1_opt, x2_opt
    else:
        raise RuntimeError("Optimization failed to converge.")

In [ ]:
def get_voids(phi):
    _, voids = measure.label(np.where(phi >= 0, 1, 0), return_num = True)
    return voids - 1

def get_genus_surface_area(edt_in, c, step_size):
    verts, faces, normals, values = measure.marching_cubes(edt_in, level = c, step_size = step_size, allow_degenerate=False)
    n = verts.shape[0]
    f = faces.shape[0]
    pmesh = pv.PolyData(verts, np.insert(faces, 0, 3, axis=1))
    e = pmesh.extract_all_edges().n_cells

    surface_area = pmesh.area

    EP = n - e + f
    g = 1 - EP/2
    return g, surface_area

def calc_csd(phi, l, nbins = 50, step_size = 4):
    phi_edt = create_euclidean_distance_transform(phi)
    voids = get_voids(phi)
    V = np.prod(phi.shape)

    g, A = get_genus_surface_area(phi_edt, 0, step_size)
    sigma = A/V
    phi_edt_norm = phi_edt*(sigma*2)
    # phi_edt_norm = np.round(phi_edt_norm, 3)
    
    radii_norm = np.linspace(0, 0.5, nbins)
    gv = np.zeros_like(radii_norm)
    hv = np.zeros_like(radii_norm)

    edt_in = convolve(phi_edt_norm.astype(float), np.ones((l, l, l)), mode = "same")/np.power(l, 3)

    for i in range(radii_norm.size):
        c = radii_norm[i]

        g, A = get_genus_surface_area(edt_in, c, step_size)
        gv[i] = g/(V*np.power(sigma, 3))
        hv[i] = (g - voids)/(V*np.power(sigma, 3))

        g, A = get_genus_surface_area(edt_in, -c, step_size)
        gv[i] = g/(V*np.power(sigma, 3))
        hv[i] = (g - voids)/(V*np.power(sigma, 3))

        gv[i] /= 2
        hv[i] /= 2
    
    return 2*radii_norm, gv, hv

def create_euclidean_distance_transform(phi):
    phi_edt = np.empty(phi.shape)

    # phi_bin_pos = schiller_lab_tools.microstructure_analysis.label_regions_hk(phi, lambda x: np.where(x >= 0, 1, 0))
    # phi_bin_neg = schiller_lab_tools.microstructure_analysis.label_regions_hk(phi, lambda x: np.where(x <= 0, 1, 0))

    phi_bin_pos = np.where(phi >= 0, 1, 0)
    phi_bin_neg = np.where(phi <= 0, 1, 0)

    phi_edt_pos = distance_transform_edt(phi_bin_pos)
    phi_edt_neg = distance_transform_edt(phi_bin_neg)

    idxs = np.where(phi >= 0)
    phi_edt[idxs] = phi_edt_pos[idxs]
    idxs = np.where(phi <= 0)
    phi_edt[idxs] = -phi_edt_neg[idxs]

    return phi_edt

## CSD - EDT

$$S_v = \frac{A_{int}}{V}$$

$S_v$ is the surface area per unit volume calculated as the ratio of the surface area $A_{int}$ and $V$ as the system volume.

In [ ]:
prep_csd = gaussian_filter(phi_fill.copy(), sigma = 1)

verts, faces, normals, values = measure.marching_cubes(prep_csd, level = 0, step_size = 1, allow_degenerate=False)

n = verts.shape[0]
f = faces.shape[0]
pmesh = pv.PolyData(verts, np.insert(faces, 0, 3, axis=1))
e = pmesh.extract_all_edges().n_cells
# e = 3/2*(f)

surface_area = pmesh.area
sigma = surface_area/(np.prod(boxDims))
# print(sigma)

EP = n - e + f
g = 1 - EP/2

_, voids = measure.label(np.where(prep_csd > 0, 1, 0), return_num = True)
handles = g - voids

In [ ]:
from scipy.ndimage import distance_transform_edt

phi_bin_pos = schiller_lab_tools.microstructure_analysis.label_regions_hk(prep_csd)
phi_bin_neg = schiller_lab_tools.microstructure_analysis.label_regions_hk(prep_csd, lambda x: np.where(x < 0, 1, 0))

phi_edt_pos = distance_transform_edt(phi_bin_pos)
phi_edt_neg = distance_transform_edt(phi_bin_neg)

phi_edt = np.zeros(boxDims)

idxs = np.where(phi_fill > 0)
phi_edt[idxs] = phi_edt_pos[idxs]
idxs = np.where(phi_fill < 0)
phi_edt[idxs] = -phi_edt_neg[idxs]

In [ ]:
fig, axs = plt.subplots(1, 3, figsize = (12, 4), layout = "constrained")

slc = np.s_[:, boxDims[1]//2, :]

ax = axs[0]
im = ax.imshow(phi_fill[slc])
plt.colorbar(im, ax = ax, orientation = "horizontal", shrink = 0.8, label = r"$\phi$")
ax.set_title("Slice of " + r"$\phi$")

ax = axs[1]
im = ax.imshow(phi_edt[slc], cmap = 'magma')

X,Y = np.indices(phi_edt[slc].shape)

CS = ax.contour(Y, X, phi_edt[slc], levels = 5, alpha = 1)
ax.clabel(CS, fontsize=10)

plt.colorbar(im, ax = ax, orientation = "horizontal", shrink = 0.8, label = "edt")
ax.set_title(r"$\phi_{edt}$" + ". Contour of \n interface from edt shown")

ax = axs[2]
temp = phi_edt.astype(int)
temp_slc = temp[slc]*(sigma*2)
im = ax.imshow(temp_slc, cmap = 'magma')

X,Y = np.indices(temp_slc.shape)

CS = ax.contour(Y, X, temp_slc, levels = 5, alpha = 1)
ax.clabel(CS, fontsize=10)

plt.colorbar(im, ax = ax, orientation = "horizontal", shrink = 0.8, label = "edt")
ax.set_title(r"$\bar{\phi}_{edt}$" + ". Contour of \n interface from edt shown")

# fig.suptitle("Slice of " + r"$\phi$" + " and Euclidean distance transform of " + r"$\phi$")
fig.savefig("csd_prep.png", dpi = 300)
# schiller_lab_tools.visualization.write_vti(".", "phi_edt", {"phi":phi_fill, "phi_edt":phi_edt.astype(int), "phi_edt_norm":np.round(phi_edt*sigma, 2)})

In [ ]:
@njit
def boxcar_avg(data, l = 1):
    # 3 cases in for loop
    #   1. Interior portions
    #   2. Edges 
    #   3. Corners
    # Take care of all cases using modulo operator
    nx, ny, nz = data.shape
    for i in range(0, nx):
        for j in range(0, ny):
            for k in range(0, nz):      
                counter = 0.0
                sum_data = 0.0
                for x in range(i-l, i+l):
                    for y in range(j-l, j+l):
                        for z in range(k-l, k+l):
                            wrap_x = x%nx
                            wrap_y = y%ny
                            wrap_z = z%ny

                            counter = counter + 1.0
                            sum_data = sum_data + data[wrap_x, wrap_y, wrap_z]

                data[i, j, k] = sum_data/counter
    
    return data

In [ ]:
phi_edt_norm = phi_edt*(sigma*2)
l = 3

edt_in_1 = boxcar_avg(phi_edt_norm.astype(float), l = 3)
edt_in_2 = convolve(phi_edt_norm.astype(float), np.ones((l, l, l)), mode = "same")

fig, axs = plt.subplots(1, 2, figsize = (8, 4))

ax = axs[0]
im = ax.imshow(edt_in_1[128])
plt.colorbar(im, ax= ax)

ax = axs[1]
im = ax.imshow(edt_in_2[128]/27)
plt.colorbar(im, ax= ax)

In [ ]:
def get_voids(phi):
    _, voids = measure.label(np.where(phi >= 0, 1, 0), return_num = True)
    return voids - 1

def get_genus_surface_area(edt_in, c, step_size):
    verts, faces, normals, values = measure.marching_cubes(edt_in, level = c, step_size = step_size, allow_degenerate=False)
    n = verts.shape[0]
    f = faces.shape[0]
    pmesh = pv.PolyData(verts, np.insert(faces, 0, 3, axis=1))
    e = pmesh.extract_all_edges().n_cells

    surface_area = pmesh.area

    EP = n - e + f
    g = 1 - EP/2
    return g, surface_area

def calc_csd(phi, l, nbins = 50, step_size = 4):
    phi_edt = create_euclidean_distance_transform(phi)
    voids = get_voids(phi)
    V = np.prod(phi.shape)

    g, A = get_genus_surface_area(phi_edt, 0, step_size)
    sigma = A/V
    phi_edt_norm = phi_edt*(sigma*2)
    # phi_edt_norm = np.round(phi_edt_norm, 3)
    
    radii_norm = np.linspace(0, 1, nbins)
    gv = np.zeros_like(radii_norm)
    hv = np.zeros_like(radii_norm)

    edt_in = convolve(phi_edt_norm.astype(float), np.ones((l, l, l)), mode = "same")/np.power(l, 3)

    for i in range(radii_norm.size):
        c = radii_norm[i]

        g, A = get_genus_surface_area(edt_in, c, step_size)
        gv[i] = g/(V*np.power(sigma, 3))
        hv[i] = (g - voids)/(V*np.power(sigma, 3))

        g, A = get_genus_surface_area(edt_in, -c, step_size)
        gv[i] = g/(V*np.power(sigma, 3))
        hv[i] = (g - voids)/(V*np.power(sigma, 3))

        gv[i] /= 2
        hv[i] /= 2
    
    return radii_norm, gv, hv

def create_euclidean_distance_transform(phi):
    phi_edt = np.empty(phi.shape)

    # phi_bin_pos = schiller_lab_tools.microstructure_analysis.label_regions_hk(phi, lambda x: np.where(x >= 0, 1, 0))
    # phi_bin_neg = schiller_lab_tools.microstructure_analysis.label_regions_hk(phi, lambda x: np.where(x <= 0, 1, 0))

    phi_bin_pos = np.where(phi >= 0, 1, 0)
    phi_bin_neg = np.where(phi <= 0, 1, 0)

    phi_edt_pos = distance_transform_edt(phi_bin_pos)
    phi_edt_neg = distance_transform_edt(phi_bin_neg)

    idxs = np.where(phi >= 0)
    phi_edt[idxs] = phi_edt_pos[idxs]
    idxs = np.where(phi <= 0)
    phi_edt[idxs] = -phi_edt_neg[idxs]

    return phi_edt

In [ ]:
import time

# phi_edt = create_euclidean_distance_transform(phi_fill)

ls_r = []
ls_gv = []
ls_hv = []

step_sizes = np.arange(2, 4, 1)
step_sizes = 2**step_sizes

for step_size in step_sizes:
    start = time.time()
    radii_norm, gv, hv = calc_csd(phi_fill, l = 3, nbins = 101, step_size = step_size)
    end = time.time()

    print(end - start)

    ls_r.append(radii_norm)
    ls_gv.append(gv)
    ls_hv.append(hv)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize = (12, 3))
colors = ["k", 'b', 'r', 'tab:orange']
smoothening = 0

for i in range(step_sizes.size):
    ax = axs[0]
    radii_norm = ls_r[i]
    gv = ls_gv[i]
    ax.plot(radii_norm, gaussian_filter(gv, sigma = smoothening), label = f"step_size = {step_sizes[i]}", color= colors[i])

    ax = axs[1]
    radii_norm = ls_r[i]
    hv = ls_hv[i]
    ax.plot(radii_norm, gaussian_filter(hv, sigma = smoothening), label = f"step_size = {step_sizes[i]}", color= colors[i])

    ax = axs[2]
    radii_norm = ls_r[i]
    hv = ls_hv[i]
    # hv_diff = np.diff(hv)
    # csd = gaussian_filter(hv_diff, sigma = smoothening)
    csd = np.diff(gaussian_filter(hv, sigma = smoothening))
    ax.plot(radii_norm[:-1], np.abs(csd), label = f"step_size = {step_sizes[i]}", color= colors[i])

ax.legend()

ax = axs[0]
ax.set_xlabel(r"$\bar{r}$")
ax.set_ylabel(r"$g_v$")

ax = axs[1]
ax.set_xlabel(r"$\bar{r}$")
ax.set_ylabel(r"$h_v$")

ax = axs[2]
ax.set_xlabel(r"$\bar{r}$")
ax.set_ylabel(r"$f$")


fig.tight_layout()

In [ ]:
import time

# phi_edt = create_euclidean_distance_transform(phi_fill)

ls_r = []
ls_gv = []
ls_hv = []

# bin_sizes = np.arange(10, 60, 10)
# bin_sizes = np.array([30, 50, 80])
bin_sizes = np.array([50])
# step_sizes = 2**step_sizes

for bin in bin_sizes:
    start = time.time()
    radii_norm, gv, hv = calc_csd(phi_fill, l = 3, nbins = bin, step_size = 2)
    end = time.time()

    print(end - start)

    ls_r.append(radii_norm)
    ls_gv.append(gv)
    ls_hv.append(hv)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize = (12, 3))
colors = ["k", 'b', 'r', 'tab:orange', 'tab:green']
markers = ['o', 's', '^']
smoothening = 2

for i in range(bin_sizes.size):
    ax = axs[0]
    radii_norm = ls_r[i]
    gv = ls_gv[i]
    ax.plot(radii_norm, gaussian_filter(gv, sigma = smoothening), label = f"step_size = {bin_sizes[i]}", 
            color= colors[i], marker = markers[i], markerfacecolor = "None", ls = "None")

    ax = axs[1]
    radii_norm = ls_r[i]
    hv = ls_hv[i]
    ax.plot(radii_norm, gaussian_filter(hv, sigma = smoothening), label = f"step_size = {bin_sizes[i]}", 
            color= colors[i], marker = markers[i], markerfacecolor = "None", ls = "None")

    ax = axs[2]
    radii_norm = ls_r[i]
    hv = ls_hv[i]

    csd = np.diff(gaussian_filter(hv, sigma = smoothening))
    ax.plot(radii_norm[:-1], np.abs(csd), label = f"step_size = {bin_sizes[i]}", 
            color= colors[i], marker = markers[i], markerfacecolor = "None", ls = "None")

# ax.legend()

ax = axs[0]
ax.set_xlabel(r"$\bar{r}$")
ax.set_ylabel(r"$g_v$")

ax = axs[1]
ax.set_xlabel(r"$\bar{r}$")
ax.set_ylabel(r"$h_v$")

ax = axs[2]
ax.set_xlabel(r"$\bar{r}$")
ax.set_ylabel(r"$f$")


fig.tight_layout()

## Curvature distribution with space

In [ ]:
verts, faces, normals, values = measure.marching_cubes(phi_fill, level = 0, step_size = 4, allow_degenerate=False)

n = verts.shape[0]
f = faces.shape[0]
pmesh = pv.PolyData(verts, np.insert(faces, 0, 3, axis=1))

In [ ]:
H = pmesh.curvature("mean")
filt = np.abs(H) < 2*(sigma)
H = H[filt]
verts_filt = verts[filt]

fig, ax = plt.subplots(1, 1, figsize = (4, 4))

counts, bins, patch = ax.hist(H, bins = 50, density = True)

bin_indices = np.digitize(H, bins)

# bin_width = 5e-4

# bins = np.arange(H.min(), H.max(), bin_width)

# np.histogram(H, bins = bins)

# H1 = H*sigma
# H2 = H1.copy()

In [ ]:
R_cut = 0.25

test = np.zeros((bin_indices.size, bin_indices.size))
H1 = test.copy()
H2 = test.copy()
binsize = 0.3*(1/sigma)

unique_bin_idx = np.unique(bin_indices)

for i in range(unique_bin_idx.size - 1):
    for j in range(i, unique_bin_idx.size):
        idx1 = i + 1
        idx2 = j + 1
        g1 = verts_filt[bin_indices == idx1]    
        g2 = verts_filt[bin_indices == idx2]

        r, gr = radial_distribution_function(256, g1, g2, binsize = binsize)
        x = r*sigma
        y = gaussian_filter(gr, sigma = 0.4)

        idx_sel = np.argsort(np.abs(x - R_cut))[0]
        
        test[i,j] = y[idx_sel]

        H1[i, j] = bins[i]
        H2[i, j] = bins[j]

plt.contourf(H1/sigma, H2/sigma, test)
plt.colorbar()

## Distance between particles

In [ ]:
Rp1 = 12.6
Rp2 = Rp1
Ro1 = 6.3
Ro2 = Ro1

In [ ]:
p1 = md.iloc[1] # 13, 1215, 1019
p2 = md.iloc[695] # 16, 1216, 1021

center1 = p1[['x', 'y', 'z']].to_numpy()
orientation1 = p1[['o_x', 'o_y', 'o_z']].to_numpy()

center2 = p2[['x', 'y', 'z']].to_numpy()
orientation2 = p2[['o_x', 'o_y', 'o_z']].to_numpy()

d, _, _= closest_ellipsoid_distance(center1, orientation1, Rp1, Ro1, center2, orientation2, Rp2, Ro2)
d

In [ ]:
npart = md.shape[0]

dists = np.zeros((npart, npart))

for i in range(npart-1):
    for j in range(i+1, npart):

        p1 = md.iloc[i] # 13, 1215, 1019
        p2 = md.iloc[j] # 16, 1216, 1021

        center1 = p1[['x', 'y', 'z']].to_numpy()
        orientation1 = p1[['o_x', 'o_y', 'o_z']].to_numpy()

        center2 = p2[['x', 'y', 'z']].to_numpy()
        orientation2 = p2[['o_x', 'o_y', 'o_z']].to_numpy()

        dists[i, j], _, _= closest_ellipsoid_distance(center1, orientation1, Rp1, Ro1, center2, orientation2, Rp2, Ro2)

## CSD - level set

In [ ]:
slc1 = np.s_[128, 128]
slc2 = np.s_[60:70]
x_s = np.linspace(0, 255, 256)

xraw = x_s[slc2]
yraw = phi_fill[slc1][slc2]

fig, axs = plt.subplots(2, 2, figsize = (8, 8), layout = 'constrained')

ax = axs[0, 0]
ax.plot(xraw, yraw, 'rx')

fit_func = lambda x, p0, hint, xi, c: p0*np.tanh((x - hint)/(np.sqrt(2)*xi)) + c
popt, pcov = curve_fit(fit_func, xraw, yraw, p0 = (0.7, np.mean(xraw), 5, 0.35))

xfit = np.linspace(xraw.min(), xraw.max(), 101)
yfit = fit_func(xfit, *popt)

ax.plot(xfit, yfit, 'b-')
ax.set_title("Original profile and fit")
print(popt)

ax = axs[0, 1]
ax.plot(xraw, np.abs(np.gradient(yraw)), 'rx')
ax.set_title("Gradient of original profile")

ax = axs[1, 0]
delta = 1/popt[2]
scale_factor = delta/2/np.mean(phi_fill[phi_fill > 0])
var_phi = phi_fill.copy()*scale_factor

xraw = x_s[slc2]
yraw = var_phi[slc1][slc2]

ax.plot(xraw, yraw, 'rx')
ax.axhline(1, xmin = 0, xmax = 1, color = 'k', ls = '--')
ax.set_title("Rescaled profile")

ax = axs[1, 1]
ax.axhline(1, xmin = 0, xmax = 1, color = 'k', ls = '--')
ax.plot(xraw, np.abs(np.gradient(yraw)), 'rx')
ax.set_title("Gradient of rescaled profile")
# var_phi = 

In [ ]:
def iterate(phi, delta = 1e-5, epsilon = 1e-1, dt = 1/3):
    sgn_phi = phi/np.sqrt(np.power(phi, 2) + np.power(epsilon, 2))
    abs_grad = np.linalg.norm(np.gradient(phi), axis = 0)
    
    phi_out = phi + sgn_phi*(1 - abs_grad)*dt
    phi_out = np.where(phi_out*phi > 0, phi_out, phi)
    return phi_out

phi_in = transform.downscale_local_mean(var_phi, (1, 1, 1)) 
phi_m0 =  phi_in.copy()
phi_m1 = np.empty(phi_m0.shape)

max_ite = 6

for i in range(0, max_ite):
    phi_m1 = iterate(phi_m0)
    phi_m0 = phi_m1.copy()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (8, 4), layout = "constrained")

slc = np.s_[:, :, 128]

X, Y, Z = np.indices(phi_in.shape)

ax = axs[0]
im = ax.imshow(phi_in[:, 128, :])
plt.colorbar(im, ax = ax)

grad = np.linalg.norm(np.gradient(phi_in), axis = 0)
ax.contour(X[slc], Y[slc], grad[slc], levels = [1], cmap = "bwr")

ax = axs[1]
im = ax.imshow(phi_m1[:, 128, :])
plt.colorbar(im, ax = ax)

grad = np.linalg.norm(np.gradient(phi_m1), axis = 0)
ax.contour(X[slc], Y[slc], grad[slc], levels = [1], cmap = "bwr")

In [ ]:
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

mesh = Poly3DCollection(verts[faces])
# mesh.set_edgecolor('k')
ax.add_collection3d(mesh)

# Visualization

In [ ]:
# x = np.random.randn(32, 30)
# y = np.random.randn(32, 30)

# ani = schiller_lab_tools.visualization.animate_plot(x, y)
# HTML(ani.to_jshtml())

In [ ]:
# test = np.random.randn(30, 32, 32)*10

# ani = schiller_lab_tools.visualization.animate_colormap(test)
# HTML(ani.to_jshtml())

# Microstructure analysis

In [ ]:
# k1 = schiller_lab_tools.microstructure_analysis.spherical_first_moment(phi_fill)
# D1 = 2*np.pi/k1
# D1

In [ ]:
# k2 = schiller_lab_tools.microstructure_analysis.second_moment(phi_fill)
# D2 = 2*np.pi/np.sqrt(k2)
# D2

In [ ]:
# S_i = schiller_lab_tools.microstructure_analysis.interface_order(phi_fill)
# S_i

In [ ]:
# filter_func = lambda x: np.where(x > 0, 1, 0)

# phi_bin = schiller_lab_tools.microstructure_analysis.label_regions_hk(phi_fill, filter=filter_func)

# fig, axs = plt.subplots(1, 2, figsize = (8, 4))

# slc = np.s_[:, boxDims[1]//2, :]

# ax = axs[0]
# im = ax.imshow(phi_fill[slc])
# ax.set_title("raw input")
# plt.colorbar(im, ax = ax, orientation = "horizontal", shrink = 0.8)

# ax = axs[1]
# im = ax.imshow(phi_bin[slc])
# ax.set_title("Binarized")
# plt.colorbar(im, ax = ax, orientation = "horizontal", shrink = 0.8)

In [ ]:
# %%time

# limit = 2/D1
# K, H, A = schiller_lab_tools.microstructure_analysis.curvature(phi_fill, (limit**2, limit), step_size = 1)

# fig, axs = plt.subplots(1, 2, figsize = (8, 4))
# a = 0.6
# b = 20

# ax = axs[0]
# ax.hist(K, alpha = a, edgecolor = "k", bins = b, density = True)
# ax.set_title("Gaussian curvature")

# ax = axs[1]
# ax.hist(H, alpha = a, edgecolor = "k", bins = b, density = True)
# ax.set_title("Mean curvature")

In [ ]:
# pn = schiller_lab_tools.microstructure_analysis.get_pn(phi_fill, parallel = {'cores':1, 'divs':[1, 1, 1], 'overlap':8})

# fig, axs = plt.subplots(2,2, figsize = (6, 6), layout = "constrained")
# a = 0.6
# b = 20

# ax = axs[0, 0]
# parameter = 'pore.equivalent_diameter' 
# ax.hist(pn[parameter], alpha = a, bins = b, density = True, edgecolor = "k")
# ax.set_title(parameter)

# ax = axs[0, 1]
# parameter = 'pore.inscribed_diameter' 
# ax.hist(pn[parameter], alpha = a, bins = b, density = True, edgecolor = "k")
# ax.set_title(parameter)

# ax = axs[1, 0]
# parameter = 'throat.equivalent_diameter' 
# ax.hist(pn[parameter], alpha = a, bins = b, density = True, edgecolor = "k")
# ax.set_title(parameter)

# ax = axs[1, 1]
# parameter = 'throat.inscribed_diameter' 
# ax.hist(pn[parameter], alpha = a, bins = b, density = True, edgecolor = "k")
# ax.set_title(parameter)

In [ ]:
# from skimage.transform import downscale_local_mean

# filter_func = lambda x: np.where(x > 0, 1, 0)
# tau_s = schiller_lab_tools.microstructure_analysis.taufactor_tortuosity(downscale_local_mean(phi_fill, (4, 4, 4)), filter = filter_func, convergence_criteria=0.1)

# Particle Analysis

In [ ]:
# positions = md[['x', 'y', 'z']].to_numpy()
# orientations = md[['o_x', 'o_y', 'o_z']].to_numpy()
# theta, not_in_interface = schiller_lab_tools.particle_analysis.calculate_average_cos_interface_normal(phi_fill, positions, orientations, step_size = 2)

# fig, ax = plt.subplots(1, 1, figsize = (4, 4))
# a = 0.6
# b = 20
# bins, counts, patch = ax.hist(theta, alpha = a, bins = b, edgecolor = "k", density = True)
# ax.set_title("Angle to interface normal")

In [ ]:
# orientations = md[['o_z', 'o_y', 'o_x']].to_numpy()
# S = schiller_lab_tools.particle_analysis.calculate_nematic_order(orientations, director = [0, 0, 1])

In [ ]:
# positions = md[['x', 'y', 'z']].to_numpy()
# r, gr = schiller_lab_tools.particle_analysis.calculate_rdf(boxDims, positions)

# fig, ax = plt.subplots(1, 1, figsize = (4, 4))
# ax.plot(r, gr, ls = "-", marker = "None", markerfacecolor = "None")
# ax.set_title("RDF")

In [ ]:
# positions = md[['x', 'y', 'z']].to_numpy()
# Q4 = schiller_lab_tools.particle_analysis.calculate_ql(boxDims, positions, L = 4)
# Q6 = schiller_lab_tools.particle_analysis.calculate_ql(boxDims, positions, L = 6)

# test = np.array([Q4, Q6])
# test = test.T
# # test = whiten(test)

# clusters = 1
# centers, mean_dist = kmeans(test, k_or_guess = clusters)

# fig, ax = plt.subplots(1, 1, figsize = (4, 4))

# ax.plot(test[:, 0], test[:, 1], ls = "None", marker = ".", markerfacecolor = "None", color = "tab:blue")
# # ax.scatter(test[:, 0], test[:, 1])
# for i in range(clusters):
#     ax.plot(centers[i,0], centers[i, 1], color = "tab:red", marker = "o", ms = 10)

In [ ]:
# positions = md[['x', 'y', 'z']].to_numpy()
# w4 = schiller_lab_tools.particle_analysis.calculate_wl(boxDims, positions, L = 4)
# w6 = schiller_lab_tools.particle_analysis.calculate_wl(boxDims, positions, L = 6)

# test = np.array([w4, w6])
# test = test.T
# # test = whiten(test)

# clusters = 1
# centers, mean_dist = kmeans(test, k_or_guess = clusters)

# fig, ax = plt.subplots(1, 1, figsize = (4, 4))
# ax.plot(test[:, 0], test[:, 1], ls = "None", marker = ".", markerfacecolor = "None")
# ax.set_xlabel("w4")
# ax.set_ylabel("w6")

# for i in range(clusters):
#     ax.plot(centers[i,0], centers[i, 1], color = "tab:red", marker = "o", ms = 10)

In [ ]:
# plt.plot(Q4, w4, ls = "None", marker = ".", color = "tab:blue", markerfacecolor = "None")
# plt.xlabel("Q6")
# plt.ylabel("w4")

In [ ]:
# 